In [39]:

import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot

# Optional for real-time simulation
# import pathway as pw 

output_notebook()


Loading BokehJS ...

In [40]:
df = pd.read_csv("dataset.csv")  # Update path if needed
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [41]:
def preprocess(df):
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])  # only convert if it exists
    
    df['occupancy_ratio'] = df['occupancy'] / df['capacity']
    
    vehicle_weight_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    df['vehicle_weight'] = df['vehicle_type'].map(vehicle_weight_map)
    
    return df


In [42]:
df.head()
print(df.columns)


Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')


In [43]:
# Example if the column is named "Occupancy" (with capital O)
df['price_model1'] = 10  # Initial base price

for i in range(1, len(df)):
    df.loc[i, 'price_model1'] = linear_price_update(
        df.loc[i-1, 'price_model1'],
        df.loc[i, 'Occupancy'],     # adjust here
        df.loc[i, 'Capacity']       # and here
    )


C:\Users\vinay singhal\AppData\Local\Temp\ipykernel_21512\1883980208.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.221837088388215' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'price_model1'] = linear_price_update(


In [44]:
df.columns = df.columns.str.lower()


In [45]:
df['occupancy']
df['capacity']


0         577
1         577
2         577
3         577
4         577
         ... 
18363    1920
18364    1920
18365    1920
18366    1920
18367    1920
Name: capacity, Length: 18368, dtype: int64

In [46]:
# Convert all column names to lowercase for consistency
df.columns = df.columns.str.lower()

# 🔧 Ensure required columns exist
required_cols = ['occupancy', 'capacity', 'queue_length', 'traffic_level', 'is_special_day', 'vehicle_type']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print("Missing columns:", missing_cols)
else:
    # Step 1: Add occupancy_ratio
    df['occupancy_ratio'] = df['occupancy'] / df['capacity']

    # Step 2: Map vehicle type to weight
    vehicle_weight_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    df['vehicle_weight'] = df['vehicle_type'].map(vehicle_weight_map).fillna(1.0)  # default to 1.0 if unknown

    # Step 3: Define demand function
    def compute_demand(row, weights):
        return (
            weights['α'] * row['occupancy_ratio'] +
            weights['β'] * row['queue_length'] -
            weights['γ'] * row['traffic_level'] +
            weights['δ'] * row['is_special_day'] +
            weights['ε'] * row['vehicle_weight']
        )

    # Step 4: Normalize demand
    def normalize(series):
        return (series - series.min()) / (series.max() - series.min())

    # Step 5: Define weights
    weights = {'α': 1.2, 'β': 0.8, 'γ': 0.6, 'δ': 1.0, 'ε': 1.5}

    # Step 6: Calculate demand and normalized demand
    df['demand'] = df.apply(lambda row: compute_demand(row, weights), axis=1)
    df['norm_demand'] = normalize(df['demand'])

    # Step 7: Calculate dynamic price
    def demand_based_price(base_price, norm_demand, λ=0.5):
        price = base_price * (1 + λ * norm_demand)
        return np.clip(price, 5, 20)  # Bound price between $5 and $20

    df['price_model2'] = df['norm_demand'].apply(lambda d: demand_based_price(10, d))

    # Done!
    print("Model 2 pricing complete. Sample prices:")
    print(df[['demand', 'norm_demand', 'price_model2']].head())


Missing columns: ['queue_length', 'traffic_level', 'is_special_day', 'vehicle_type']


In [47]:
# 📌 Section 6: Model 3 (Optional) - Competitive Model
def haversine(lat1, lon1, lat2, lon2):
    # Distance in km between 2 geo points
    R = 6371
    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

# Add logic to compute distances between lots and adjust pricing accordingly


In [48]:
from bokeh.plotting import figure, show

def plot_prices(df, model='price_model2'):
    # Check if the required columns exist in the DataFrame
    required_columns = ['parking_lot_id', 'timestamp', model]
    for col in required_columns:
        if col not in df.columns:
            print(f"Error: Column '{col}' is not in the DataFrame.")
            return

    p = figure(title="Dynamic Pricing Over Time", x_axis_label="Time", y_axis_label="Price")
    for lot_id in df['parking_lot_id'].unique():
        lot_data = df[df['parking_lot_id'] == lot_id]
        p.line(lot_data['timestamp'], lot_data[model], legend_label=f"Lot {lot_id}", line_width=2)
    
    p.legend.click_policy = "hide"
    show(p)

# Call the function with your DataFrame
plot_prices(df)


Error: Column 'parking_lot_id' is not in the DataFrame.
